In [1]:
import json
from itertools import chain
import os.path as osp
import torch
from torch.utils.data import DataLoader
from torchvision import transforms as T
from tqdm.auto import tqdm

from config import cfg, hrnet_cfg
from src.datasets.dataset import DatasetReID
from src.models.orientations.pose_hrnet import PoseHighResolutionNet
from utils.misc import get_filename, make_objects

In [2]:
transform = T.Compose([
    T.Resize(size=(256, 192)),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

model = PoseHighResolutionNet(hrnet_cfg)
with open(cfg.MEBOW.PRETRAINED, "rb") as f:
    weights = torch.load(f, map_location='cpu')
model.load_state_dict(weights)
model.eval()
model.cuda()


PoseHighResolutionNet(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
  (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(i

In [38]:
mode = 0 #training
# dataset_names = ['market1501', 'cuhk03', 'msmt17', 'viper', 'ltcc', 'prcc', 'vc_clothes', 'real28']
# for dataset_name in dataset_names:
dataset_name = 'vc-clothes'
dataset = DatasetReID(dataset_name, transforms=transform)
loader = DataLoader(dataset=dataset, batch_size = cfg.MEBOW.BATCH_SIZE)

In [39]:
len(dataset)

9449

In [40]:
out = []

with torch.inference_mode():
    for x, _ in tqdm(loader):
        x = x.cuda()
        _, hoe = model(x)
        orient = torch.argmax(hoe, dim=1)
        out.append(orient.cpu().view(1, -1)[0])

100%|██████████| 296/296 [01:04<00:00,  4.57it/s]


In [41]:
out = list(map(lambda s: s.numpy(), out))
out = list(chain.from_iterable(out))
out = list(map(lambda s: int(s), out))

In [42]:
with open(osp.join('data', dataset_name, 'jsons/train.json'), "rb") as f:
        data = json.load(f)

data
for i, img in enumerate(data):
        img["orientation"] = out[i]
data

with open(osp.join('data', dataset_name, 'jsons/train.json'), "w") as f:
        json.dump(data, f, indent=2)